In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import re

# ------------------------
# 1. CSV Dosyasını Yükleme
# ------------------------
df = pd.read_csv("/home/yapayzeka/ahsen_bulbul/data/cleaned10chunk.csv")

# ------------------------
# 2. Temizleme Fonksiyonu
# ------------------------
def clean_text(text):
    # Gereksiz üç nokta ve fazla boşlukları kaldır
    text = text.replace("...", " ")
    text = re.sub(r'\s+', ' ', text)  # Çoklu boşluk → tek boşluk
    text = text.strip()
    
    # İsteğe bağlı: bazı noktalama işaretlerini kaldır
    #text = re.sub(r'[;,:-]', '', text)
    
    return text

# ------------------------
# 3. Chunking Parametreleri
# ------------------------
CHUNK_SIZE = 400      # kelime bazlı
OVERLAP = 200         # kelime bazlı

# ------------------------
# 4. Semantic Chunking + Sliding Window
# ------------------------
all_chunks = []


for idx, row in df.iterrows():
    text = clean_text(row['rawText'])  # Temizleme adımı
    words = text.split()  # kelime bazlı tokenizasyon
    start = 0
    while start < len(words):
        end = start + CHUNK_SIZE
        chunk_text = " ".join(words[start:end])
        
        # Metadata ekle
        chunk_data = {
            "chunk_id": f"{row['_id']}_{start}",
            "text": chunk_text,
            "mahkeme": row['location'],
            "karar_id": row['_id']
        }
        all_chunks.append(chunk_data)
        
        # Sliding window
        start += CHUNK_SIZE - OVERLAP

# ------------------------
# 5. Embedding Oluşturma
# ------------------------
model = SentenceTransformer('all-mpnet-base-v2')  # 768 boyut


chunk_texts = [c['text'] for c in all_chunks]
embeddings = model.encode(chunk_texts, batch_size=32, show_progress_bar=True)

# ------------------------
# 6. Kontrol
# ------------------------
print(all_chunks[0])
print(len(embeddings[0]))  # 384 boyut
print(embeddings[0][:10])  # ilk 10 boyut örnek


#### Chonkie

In [ ]:
from chonkie import SemanticChunker, RecursiveChunker, RecursiveRules

In [ ]:
text = "6. Hukuk Dairesi 2023/596 E. , 2024/257 K. \n ""İçtihat Metni"" MAHKEMESİ : ... Bölge Adliye Mahkemesi 13. Hukuk Dairesi Taraflar arasındaki rücuen tazminat davasından dolayı yapılan yargılama sonunda, İlk Derece Mahkemesince davanın kabulüne karar verilmiştir. Kararın davacı vekili tarafından istinaf edilmesi üzerine, Bölge Adliye Mahkemesince başvurunun esastan reddine karar verilmiştir. Bölge Adliye Mahkemesi kararı davalı ... Grup Sosyal Hizmetleri İnş. Tem. Taah. Tic. A.Ş. vekili tarafından temyiz edilmekle; yapılan ön inceleme sonucunda gereği düşünüldü: İlk Derece Mahkemesi kararını istinaf etmeyen tarafın, Bölge Adliye Mahkemesinin istinaf başvurusunun esastan reddine ilişkin kararını temyiz etme hakkı bulunmadığından adı geçen davalı vekilinin temyiz dilekçesinin reddine karar verilmiştir. KARAR Açıklanan sebeple; Davalı ... Grup Sosyal Hizmetleri İnş. Tem. Taah. Tic. A.Ş. vekilinin temyiz dilekçesinin REDDİNE, Dosyanın İlk Derece Mahkemesine, kararın bir örneğinin kararı veren Bölge Adliye Mahkemesine gönderilmesine 18.01.2024 tarihinde oy birliğiyle karar verildi."

In [ ]:
len(text)

#### Semantic Chunker

In [ ]:
semantic_chunker = SemanticChunker(
    embedding_model="BAAI/bge-m3",
    threshold=0.9,                               # Similarity threshold (0-1) or (1-100) or "auto"
    chunk_size=64,                              # Maximum tokens per chunk
    min_sentences=3                              # Initial sentences per chunk
)

semantic_chunks = semantic_chunker.chunk(text)

for chunk in semantic_chunks:
    print(f"Chunk text: {chunk.text}")
    print(f"Token count: {chunk.token_count}")
    print(f"Number of sentences: {len(chunk.sentences)}")

In [ ]:
semantic_chunker = SemanticChunker(
    threshold=0.9,                               # Similarity threshold (0-1) or (1-100) or "auto"
    chunk_size=64,                              # Maximum tokens per chunk
    min_sentences=3                              # Initial sentences per chunk
)

semantic_chunks = semantic_chunker.chunk(text)

for chunk in semantic_chunks:
    print(f"Chunk text: {chunk.text}")
    print(f"Token count: {chunk.token_count}")
    print(f"Number of sentences: {len(chunk.sentences)}")

#### Recursive Chunker

In [ ]:
recursive_chunker = RecursiveChunker.from_recipe("markdown", lang="en")


recursive_chuns = recursive_chunker.chunk(text)

for chunk in recursive_chuns:
    print(f"Chunk text: {chunk.text}")
    print(f"Token count: {chunk.token_count}")

In [ ]:
text2="6. Hukuk Dairesi 2022/3281 E. , 2024/117 K. \n ""İçtihat Metni"" MAHKEMESİ :Asliye Hukuk Mahkemesi Taraflar arasındaki tazminat davasından dolayı yapılan yargılama sonunda İlk Derece Mahkemesince davanın reddine karar verilmiştir. İlk Derece Mahkemesi kararı davacılar vekilince temyiz edilmekle; kesinlik, süre, temyiz şartı ve diğer usul eksiklikleri yönünden yapılan ön inceleme sonucunda, temyiz dilekçesinin kabulüne karar verildikten ve Tetkik Hâkimi tarafından hazırlanan rapor dinlendikten sonra dosyadaki belgeler incelenip gereği düşünüldü: I. DAVA 1.Davacı vekili dava dilekçesinde özetle; müvekkillerinin arsa sahibi olduğunu, ...'ın müteahhit olarak işbu arsa üzerine yapmış olduğu binanın ... 1. Asliye Hukuk Mahkemesinin 2009/139 D. ... dosyası üzerinden ihtiyati tedbir konularak inşaatın durdurulduğunu, HUMK'un 109. maddesi gereğince ihtiyati tedbir kararının verildiği tarihten itibaren 10 gün içerisinde esas hakkında davanın açılması gerektiğini ve bu durumun dosyaya ibrazı gerekirken davanın açılmadığını, Hukuk Usulü Muhakemeleri Kanununun 110. maddesi, ihtiyati tedbir kararının haksız olduğunun belirlenmesi halinde tedbir kararı yüzünden uğranılan zararın tazminini düzenlediğini, ihtiyati tedbir kararını icra ettiren tarafın yasal sürede dava açmaması halinde ihtiyati tedbirin haksız konulduğunun kabulü gerektiği, kaldı ki süresinde dava açsa da durumun değişmeyeceğini belirterek müvekkillerinin inşaatının geç bitirilmesinden kaynaklı 10.000,00 TL maddi tazminatın tahsiline karar verilmesini talep etmiştir. 2.Davacı vekili duruşmadaki beyanında; tedbir sebebiyle bağımsız bölümlerinin geç teslim edileceğini, bundan kaynaklı doğacak zararları talep ettiklerini beyan etmiştir. II. CEVAP Davalı vekili cevap dilekçesinde özetle; ekonomik nedenlerle 10 gün içerisinde dava açamadıklarını, 19 gün sonra açtıkları davanın ... 4. Asliye Hukuk Mahkemesinin 2010/27 Esas sayılı sırasında kayıtlı olduğunu belirterek davanın reddini talep etmiştir. III. MAHKEME KARARI Mahkemenin 30.12.2011 tarihli ve 2010/695 Esas, 2011/916 Karar sayılı kararı ile ... 1.Asliye Hukuk Mahkemesinin 2009/139 D.... sayılı dosyası üzerinden 26.06.2009 tarihinde tedbir talep edildiği, yapılan keşif sonucu bilirkişilerin raporlarını ibraz ettikleri, 25.12.2009 tarihinde 2776 Parsel üzerinde yapımına devam edilen inşaatın tedbiren durdurulmasına karar verildiği, kararın infazı için 04.01.2010 tarihinde ... Belediye Başkanlığı'na ve Nöbetçi İcra Müdürlüğüne ayrı ayrı müzekkere yazıldığı, 05.01.2010 tarih saat 08:50 itibari ile inşaatın mühürlenerek tedbirin infaz edildiği, dolayısı ile tedbir kararın verildiği tarihten 11 gün geçtikten sonra kararın infaz edildiği, Mahkemenin infaz için yazdığı yazının 10 günlük süre içerisinde olduğu ancak infazın 11. gün gerçekleştiği, 10 günlük süre içerisinde herhangi bir dava açılmadığı, dolayısı ile HUMK'un 109. maddesi gereğince ihtiyati tedbirin infaz tarihi itibari ile kendiliğinden kalkmış olduğu, buna rağmen tedbirin infaz edildiği, HUMK'un 109. maddesi ""Aksi takdirde ihtiyati tedbir bir gûna merasime hacet kalmaksızın kendiliğinden kalkar ve iktizasına göre vazolunan tedbirin fiilen kaldırılması ihtiyati tedbiri tatbik eden daire veya memurdan talep olunabilir"" hükmünü düzenlediği, süresinden bir gün sonra tedbirin infazı sırasında davacıların yahut müteahhidin infaz memurlarından tedbir kararını isteyip süresinde dava açılıp açılmadığına dair de belgeyi istemeleri gerekirken bunu yapmadıkları, kaldı ki 05/01/2010 tarihinden hemen sonra aynı gün tedbirin verildiği dosyanın incelenerek dava açılıp açılmadığının tespit edilerek tedbirin kaldırılmasını talep etmeleri gerekirken infazdan 7 gün sonra 12.01.2010 tarihinde tedbirin kaldırılmasını talep ettikleri, böylece bir zarar varsa zararın ortaya çıkmasına ve artmasına bizzat mülk sahiplerinin ve müteahhidin sebep olduğu, zira inşaat durdurma zabtının altında... isimli şahsın imzasının olduğu, bu şahsın da inşaatı yapan müteahhit...'ın akrabası olduğu görülmekle açılan davanın reddine karar verilmiştir. IV. BOZMA VE BOZMADAN SONRAKİ YARGILAMA SÜRECİ A. Bozma Kararı 1.Mahkemenin yukarıda belirtilen kararına karşı süresi içinde davacılar vekili temyiz isteminde bulunmuştur. 2.Yargıtay (Kapatılan) 23. Hukuk Dairesinin 04.06.2013 tarihli ve 2013/2984 Esas, 2013/3773 Karar sayılı kararıyla dava tarihinde yürürlükte olan 1086 sayılı HUMK'un 109. maddesinin ''İhtiyati tedbir kararı dava ikamesinden evvel verilmiş ise tatbik edilmiş olsun olmasın kararın verildiği tarihten itibaren on gün zarfında esas hakkında dava ikamesi lazımdır. Bu müddette müddei, davasını ikame eylediğini müsbit evrakı, kararı tatbik eden memura ibrazla dosyaya vaz ve kaydettirerek mukabilinde ilmuhaber almaya mecburdur. Aksi takdirde ihtiyati tedbir bir gûna merasime hacet kalmaksızın kendiliğinden kalkar ve iktizasına göre vazolunan tedbirin fiilen kaldırılması, ihtiyati tedbiri tatbik eden daire veya memurdan talep olunabilir'' hükmünü içerdiği, somut olayda, davalının talebi üzerine ... 1. Asliye Hukuk Mahkemesinin 2009/139 D. ... sayılı dosyası üzerinden 25.12.2009 tarihinde inşaatın durdurulması yönünde ihtiyati tedbir kararı verildiği, teminatın 31.12.2009 tarihinde yatırıldığı, 04.01.2010 tarihinde mahkemece, İcra Müdürlüğüne ve Belediye Başkanlığına ihtiyati tedbir kararı gereğince işlem yapılması için müzekkereler yazıldığı, tedbir kararının 05.01.2010 günü saat 08:50'de infaz edildiği, aleyhine ihtiyati tedbir kararı verenlerce 08.01.2010 tarihinde ihtiyati tedbire itiraz edildiği, davalı tarafça 13.01.2010 tarihinde işbu dosyanın davacıları aleyhine ... 4. Asliye Hukuk Mahkemesinin 2010/27 Esas sayısına kayıtlı davanın açıldığı anlaşılmıştır. Buna göre HUMK'un 109. maddesi hükmüne aykırı olarak ihtiyati tedbire ilişkin karar tarihinden itibaren 10 gün içinde dava açılmamış olduğu, haksız ihtiyati tedbirden dolayı olan sorumluluğun kusursuz sorumluluk olduğu, yani, haksız ihtiyati tedbir koydurtmuş olan tarafın, bundan doğan maddi zararla sorumlu tutulabilmesi için, ihtiyati tedbiri kötüniyetle istemiş ve koydurmuş olması veya bunda herhangi bir ihmalinin bulunmasının şart olmadığı, ihtiyati tedbir haksız ve bundan da bir zarar doğmuş ise, bu haksız ihtiyati tedbiri koydurtmuş olan tarafın kusurlu olmasa bile bundan zarar gören karşı tarafa veya üçüncü kişiye tazminat ödemekle yükümlü olduğu, ihtiyati tedbir isteyen ve bu kararı icra ettiren kişinin, ihtiyati tedbiri icra ettirdiği halde, HUMK'un 109. maddesindeki on günlük süre içerisinde esas hakkında dava açmazsa, ihtiyati tedbirin haksız konulmuş sayılacağı, haksız ihtiyati tedbirden dolayı tazminat davası açan davacının ödenmesini istediği zararı ile haksız ihtiyati tedbir arasında uygun illiyet (nedensellik) bağı (sebep sonuç ilişkisi) bulunması da gerektiği, bu durumda, somut olayda mahkemece alınan ve icra ettirilen ihtiyati tedbir kararından sonra süresinde dava açılmamış olması nedeniyle tedbirin haksız olduğu kabul edilerek davacılara maddi zararlarını ve illiyet bağını ispat imkanı tanınmalı ve sonucuna göre karar verilmesi gerekçesi ile kararın bozulmasına karar verilmiştir. B. Mahkemece Bozmaya Uyularak Verilen Karar Mahkemenin yukarıda tarih ve sayısı belirtilen kararı ile davacı tarafın iddiaları, davalı tarafın savunmaları ... 1. Asliye Hukuk Mahkemesinin 2009/139 D. ... sayılı dosyası ile dosya kapsamı birlikte değerlendirildiğinde; davalının talebi üzerine mahkemenin 25/12/2009 tarihinde dava konusu 2776 parsel üzerinde yapımı devam edilen inşaatın tedbiren durdurulmasına karar verildiği, ... Belediye Başkanlığına kararın infazı için müzekkere yazıldığı ve 05/01/2010 tarih saat 08:50 itibari ile İnşaatın mühürlenmesi suretiyle kararın infaz edildiği, davalı HUMK'un 109.maddesinde belirtilen tedbir kararının verilmesinden itibaren 10 gün içinde esas hakkında dava açmadığından, 05/01/2010 tarihi itibariyle ihtiyati tedbir kararının kendiliğinden ortadan kalktığı ve ihtiyati tedbir kararının haksız olduğunun kabulü gerektiği her ne kadar davacı tarafça tedbirin 14/01/2010 tarihinde saat 9:00 da fiilen kaldırıldığı bozma ilamına uyulmasına karar verilmesi ile usuli kazanılmış hak doğduğunu, zararın en az kira gelir kaybı kadar olduğu belirtilmiş ise de tazminat davalısının ödemekle yükümlü olduğu miktarın uyulmasına karar verilen Yargıtay ilamında da belirtildiği üzer ihtiyati tedbir kararının icra edildiği tarih ile ihtiyati tedbirin kalktığı ya da kalkmış sayıldığı tarih arasındaki zarar olduğu, ihtiyati tedbir kararının kalkmış sayıldığı tarih ile icra edildiği tarihin aynı olduğu, arada süre zarfı bulunmaması nedeniyle yapılması talep edilen keşifte yoksun kalınan kira kaybına ilişkin hesaplama yapılamayacağından yargılamaya bir katkı sağlamayacağı kanaati ile keşif yapılmadan belirtilen süre zarfından davacının uğradığı zarar bulunmadığından davanın reddine ilişkin aşağıdaki gibi hüküm kurulmuştur. V. TEMYİZ A. Temyiz Yoluna Başvuranlar Mahkemenin yukarıda belirtilen kararına karşı süresi içinde davacılar vekili temyiz isteminde bulunmuştur. B. Temyiz Sebepleri Davacılar vekili temyiz dilekçesinde özetle; bozmaya uyularak devam eden yargılama sonucunda mahkemece tedbirin kendiliğinde kalmış sayıldığı, bu durumda uygulanan tedbirin hükümsüz olduğu ve tedbirli geçmiş bir süre olmadığı yönlü gerekçe ile davanın reddien karar verildiği, tedbirin fiilen uygulanması ile zarar oluştuğu, mahkemece verilen kararın hem bozma ilamına, hem bozmaya uyulmuş olmakla usuli müktesep haklara, hem de kanuna aykırı olduğu, tedbirin 10 günlük süre geçtikten sonra kalktığı, ancak 11 gün sonra tedbirin infaz edildiği belirtilerek kararın bozulmasına karar verilmesini talep etmiştir. C. Gerekçe 1. Uyuşmazlık ve Hukuki Nitelendirme Uyuşmazlık, arsa payı karşılığı inşaat sözleşmesinden kaynaklanan ihtiyati tedbir kararının uygulanmasından doğan zararın tazmini istemine ilişkindir. 2. İlgili Hukuk 6100 sayılı Hukuk Muhakemeleri Kanununun geçici 3 üncü maddesinin ikinci fıkrası atfıyla uygulanmasına devam olunan mülga 1086 sayılı Hukuk Usulü Muhakemeleri Kanununun 428 inci, 439 uncu maddesinin ikinci fıkrası ve 109. maddeleri, 6098 sayılı Türk Borçlar Kanununun 470 vd. maddeleri. 3. Değerlendirme Mahkemece, tazminat davalısının ödemekle yükümlü olduğu miktarın uyulmasına karar verilen Yargıtay ilamında da belirtildiği üzer ihtiyati tedbir kararının icra edildiği tarih ile ihtiyati tedbirin kalktığı ya da kalkmış sayıldığı tarih arasındaki zarar olduğu, ihtiyati tedbir kararının kalkmış sayıldığı tarih ile icra edildiği tarihin aynı olduğu, arada süre zarfı bulunmaması nedeniyle yapılması talep edilen keşifte yoksun kalınan kira kaybına ilişkin hesaplama yapılamayacağından yargılamaya bir katkı sağlamayacağı kanaatine varılarak davanın reddine karar verildiği, somut olayda; 05.01.2010 tarihinde tedbirin icra edildiği, 14.01.2010 tarihinde ise kaldırıldığı anlaşılmaktadır. Bu durumda mahkemece ihtiyati tedbir kararının kalkmış sayıldığı tarih ile icra edildiği tarihin aynı olduğuna dair karar hatalı olup, 05.01.2010 tarihi ile 14.01.2010 tarihleri arasında davacının maddi zararlarını ve illiyet bağını ispat imkanı tanınması ve sonucuna göre karar verilmesi gerekirken, davanın reddine karar verilmesi hatalı olmuş, kararın bozulmasına karar verilmiştir. VI. KARAR Açıklanan nedenlerle; Davacılar vekilinin temyiz itirazının kabulü ile temyiz olunan mahkeme kararının davacılar yararına BOZULMASINA, Peşin alınan harcın istek halinde temyiz eden davacılara iadesine, Kararın tebliğinden itibaren 15 gün içerisinde karar düzeltme kanun yolu açık olmak üzere, Dosyanın Mahkemesine gönderilmesine, 11.01.2024 tarihinde oy birliğiyle karar verildi.","30.12.2011,26.06.2009,25.12.2009,04.01.2010,05.01.2010,12.01.2010,04.06.2013,25.12.2009,31.12.2009,04.01.2010,05.01.2010,08.01.2010,13.01.2010,05.01.2010,14.01.2010,05.01.2010,14.01.2010,11.01.2024"


In [ ]:
print(type(text2))

if isinstance(text2, tuple):
    text2 = " ".join(text2)
# import pandas as pd

# df = pd.read_csv("/home/yapayzeka/ahsen_bulbul/data/yargitay.csv")
# text2 = df["rawText"]
# text2 = df["rawText"].iloc[1]

#### Langchain Chu

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-m3")

embeddings = model.encode(text)


semantic_chunk = SemanticChunker(
embeddings = embeddings,
buffer_size=30,
breakpoint_threshold_type="gradient",
breakpoint_threshold_amount=0.85,
number_of_chunks=128,
min_chunk_size=50,
)

chunks = semantic_chunk.split_text(text)
print(chunks)

# for chunk in chunks:
#     print(f"Chunk text: {chunk.text}")
#     print(f"Token count: {chunk.token_count}")
#     print(f"Number of sentences: {len(chunk.sentences)}")

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

# LangChain embedding objesi
emb_model = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")


semantic_chunk = SemanticChunker(
    embeddings=emb_model,   
    buffer_size=35,
    breakpoint_threshold_type="gradient",
    breakpoint_threshold_amount=0.7,
    number_of_chunks=200,
    min_chunk_size=200,
)

chunks = semantic_chunk.split_text(text2)  # text burada string olmalı

for i, chunk in enumerate(chunks, start=1):
    print(f"Chunk {i}: {chunk}")
    print(f"Token count: {len(chunk.split())}")
    print(f"Number of sentences: {len(chunk.split('. '))}")
    print("---")



In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

# LangChain embedding objesi
emb_model = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")

# GÜNCELLENMIŞ PARAMETRELER - Daha iyi sonuçlar için
semantic_chunk = SemanticChunker(
    embeddings=emb_model,
    buffer_size=60,                    # 40→60: Daha geniş bağlam
    breakpoint_threshold_type="gradient",
    breakpoint_threshold_amount=0.7,   # 0.45→0.7: Daha az bölme
    number_of_chunks=100,              # 150→100: Daha az chunk hedefi
    min_chunk_size=400,               # 250→400: Daha büyük minimum boyut
)

chunks = semantic_chunk.split_text(text2)  # text burada string olmalı

for i, chunk in enumerate(chunks, start=1):
    print(f"Chunk {i}: {chunk}")
    print(f"Token count: {len(chunk.split())}")
    print(f"Number of sentences: {len(chunk.split('. '))}")
    print("---")

# EK: Chunk kalitesini kontrol etmek için
print(f"\n=== ÖZET ===")
print(f"Toplam chunk sayısı: {len(chunks)}")
print(f"Ortalama token sayısı: {sum(len(chunk.split()) for chunk in chunks) / len(chunks):.1f}")

# Çok küçük chunk'ları tespit et
small_chunks = [i for i, chunk in enumerate(chunks, 1) if len(chunk.split()) < 60]
if small_chunks:
    print(f"⚠️ Küçük chunk'lar (60 tokenin altı): {small_chunks}")

# Çok büyük chunk'ları tespit et
large_chunks = [i for i, chunk in enumerate(chunks, 1) if len(chunk.split()) > 200]
if large_chunks:
    print(f"⚠️ Büyük chunk'lar (200 tokenin üstü): {large_chunks}")

# Bölüm başlıklarını kontrol et
section_chunks = []
for i, chunk in enumerate(chunks, 1):
    if any(section in chunk for section in ['I. DAVA', 'II. CEVAP', 'III. MAHKEME', 'IV. BOZMA', 'V. TEMYİZ', 'VI. KARAR']):
        section_chunks.append(i)

print(f"✓ Bölüm başlığı içeren chunk'lar: {section_chunks}")


In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

# LangChain embedding objesi
emb_model = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")

# METADATA DESTEKLI CHUNKING
semantic_chunk = SemanticChunker(
    embeddings=emb_model,
    buffer_size=60,                    
    breakpoint_threshold_type="gradient",
    breakpoint_threshold_amount=0.7,   
    number_of_chunks=100,              
    min_chunk_size=400,               
)
df = pd.read_csv("/home/yapayzeka/ahsen_bulbul/data/cleaned10chunk.csv")  # CSV yolunu kendine göre ayarla
# CSV'de 'rawText' veya metin sütunu neyse onu kullan
text2 = "\n".join(df['rawText'].astype(str).tolist())
chunks = semantic_chunk.split_text(text2)

# CHUNK'LARI METADATA İLE BİRLİKTE YÖNET
def process_legal_chunks_with_metadata(chunks):
    """Chunk'ları metadata ile birlikte işle"""
    
    processed_chunks = []
    metadata_chunk = None
    
    for i, chunk in enumerate(chunks):
        # Tarihler chunk'ını tespit et (sadece tarih/rakam içeren)
        word_count = len(chunk.split())
        
        if (word_count <= 5 and 
            all(any(c.isdigit() for c in word) or word in [',', '.'] 
                for word in chunk.split())):
            
            # Bu metadata chunk'ı
            metadata_chunk = {
                'type': 'dates_metadata',
                'content': chunk,
                'dates': chunk.split(','),
                'date_count': len(chunk.split(',')),
                'description': 'Dava sürecindeki kritik tarihler'
            }
            print(f"📅 METADATA CHUNK tespit edildi:")
            print(f"   Tarih sayısı: {metadata_chunk['date_count']}")
            print(f"   İçerik: {chunk[:100]}...")
            
        else:
            # Normal content chunk'ı
            content_info = analyze_chunk_content(chunk, i+1)
            processed_chunks.append({
                'chunk_id': len(processed_chunks) + 1,
                'content': chunk,
                'word_count': word_count,
                'type': content_info['type'],
                'key_elements': content_info['key_elements'],
                'legal_refs': content_info['legal_refs']
            })
    
    return processed_chunks, metadata_chunk

def analyze_chunk_content(chunk, chunk_num):
    """Chunk içeriğini analiz et ve kategorize et"""
    
    chunk_lower = chunk.lower()
    
    # Chunk tipini belirle
    if 'i. dava' in chunk_lower or 'dava dilekçesi' in chunk_lower:
        chunk_type = 'dava_süreci'
    elif 'ii. cevap' in chunk_lower or 'cevap dilekçesi' in chunk_lower:
        chunk_type = 'savunma'
    elif 'iii. mahkeme' in chunk_lower or 'mahkemenin' in chunk_lower:
        chunk_type = 'mahkeme_kararı'
    elif 'bozma' in chunk_lower or 'temyiz' in chunk_lower:
        chunk_type = 'temyiz_süreci'
    elif 'vi. karar' in chunk_lower or 'açıklanan nedenlerle' in chunk_lower:
        chunk_type = 'final_karar'
    else:
        chunk_type = 'genel_içerik'
    
    # Anahtar elemanları tespit et
    key_elements = []
    if 'humk' in chunk_lower:
        key_elements.append('HUMK_referansı')
    if 'ihtiyati tedbir' in chunk_lower:
        key_elements.append('ihtiyati_tedbir')
    if 'tazminat' in chunk_lower:
        key_elements.append('tazminat_talebi')
    if any(date_pattern in chunk for date_pattern in ['.2009', '.2010', '.2011', '.2013', '.2024']):
        key_elements.append('tarih_referansları')
    
    # Hukuki referansları bul
    legal_refs = []
    import re
    
    # Kanun maddesi referansları
    madde_pattern = r'(\d+)\.\s*madde'
    maddeler = re.findall(madde_pattern, chunk_lower)
    if maddeler:
        legal_refs.extend([f"madde_{m}" for m in maddeler])
    
    # Mahkeme dosya numaraları
    dosya_pattern = r'(\d{4}/\d+)'
    dosyalar = re.findall(dosya_pattern, chunk)
    if dosyalar:
        legal_refs.extend([f"dosya_{d}" for d in dosyalar])
    
    return {
        'type': chunk_type,
        'key_elements': key_elements,
        'legal_refs': legal_refs
    }

# KULLANIM VE GÖRÜNTÜLEME
content_chunks, dates_metadata = process_legal_chunks_with_metadata(chunks)

print(f"\n{'='*60}")
print(f"CHUNK ANALIZ RAPORU")
print(f"{'='*60}")

# Metadata bilgisi
if dates_metadata:
    print(f"\n📅 METADATA:")
    print(f"   Tip: {dates_metadata['type']}")
    print(f"   Tarih sayısı: {dates_metadata['date_count']}")
    print(f"   Açıklama: {dates_metadata['description']}")

# Content chunk'ları
print(f"\n📄 İÇERİK CHUNK'LARI ({len(content_chunks)} adet):")

for chunk_info in content_chunks:
    print(f"\n--- CHUNK {chunk_info['chunk_id']} ---")
    print(f"Tip: {chunk_info['type']}")
    print(f"Kelime sayısı: {chunk_info['word_count']}")
    print(f"Anahtar öğeler: {', '.join(chunk_info['key_elements']) if chunk_info['key_elements'] else 'Yok'}")
    print(f"Hukuki referanslar: {', '.join(chunk_info['legal_refs']) if chunk_info['legal_refs'] else 'Yok'}")
    print(f"İçerik önizleme: {chunk_info['content'][:150]}...")

# METADATA'YI KULLANMA ÖRNEKLERİ
def get_case_timeline_from_metadata(dates_metadata):
    """Metadata'dan dava zaman çizelgesini çıkar"""
    if not dates_metadata:
        return []
    
    dates = [date.strip() for date in dates_metadata['content'].split(',')]
    
    # Tarihleri sırala ve kategorize et
    timeline = []
    for date in dates:
        if date:
            timeline.append({
                'date': date,
                'formatted': f"{date[:2]}.{date[2:4]}.{date[4:8]}" if len(date) == 10 else date,
                'category': 'dava_süreci'  # Daha detaylı kategorizasyon yapılabilir
            })
    
    return timeline

def create_searchable_index(content_chunks, dates_metadata):
    """Arama için indeks oluştur"""
    
    search_index = {
        'metadata': dates_metadata,
        'chunks_by_type': {},
        'chunks_with_dates': [],
        'chunks_with_legal_refs': []
    }
    
    # Tip bazında grupla
    for chunk in content_chunks:
        chunk_type = chunk['type']
        if chunk_type not in search_index['chunks_by_type']:
            search_index['chunks_by_type'][chunk_type] = []
        search_index['chunks_by_type'][chunk_type].append(chunk)
        
        # Tarih içeren chunk'lar
        if 'tarih_referansları' in chunk['key_elements']:
            search_index['chunks_with_dates'].append(chunk)
        
        # Hukuki referans içeren chunk'lar
        if chunk['legal_refs']:
            search_index['chunks_with_legal_refs'].append(chunk)
    
    return search_index

# İndeks oluştur
search_index = create_searchable_index(content_chunks, dates_metadata)

print(f"\n📊 ARAMA İNDEKSİ:")
print(f"Chunk tipleri: {list(search_index['chunks_by_type'].keys())}")
print(f"Tarihli chunk'lar: {len(search_index['chunks_with_dates'])}")
print(f"Hukuki referanslı chunk'lar: {len(search_index['chunks_with_legal_refs'])}")

# ÖRNEK SORGULAR
def query_chunks(search_index, query_type, query_value=None):
    """Chunk'larda arama yap"""
    
    if query_type == 'by_type':
        return search_index['chunks_by_type'].get(query_value, [])
    
    elif query_type == 'with_dates':
        return search_index['chunks_with_dates']
    
    elif query_type == 'with_legal_refs':
        return search_index['chunks_with_legal_refs']
    
    elif query_type == 'metadata':
        return search_index['metadata']
    
    return []

# Kullanım örnekleri:
print(f"\n🔍 ÖRNEK SORGULAR:")
print(f"Mahkeme kararı chunk'ları: {len(query_chunks(search_index, 'by_type', 'mahkeme_kararı'))}")
print(f"Tarihli chunk'lar: {len(query_chunks(search_index, 'with_dates'))}")
print(f"Metadata: {query_chunks(search_index, 'metadata')['type'] if query_chunks(search_index, 'metadata') else 'Yok'}")

In [3]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
import pandas as pd

df = pd.read_csv("/home/yapayzeka/ahsen_bulbul/data/cleaned10chunk.csv")  # CSV yolunu kendine göre ayarla
# CSV'de 'rawText' veya metin sütunu neyse onu kullan
text2 = "\n".join(df['rawText'].astype(str).tolist())
# LangChain embedding objesi
emb_model = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")

# METADATA DESTEKLI CHUNKING
semantic_chunk = SemanticChunker(
    embeddings=emb_model,
    buffer_size=60,                    
    breakpoint_threshold_type="gradient",
    breakpoint_threshold_amount=0.7,   
    number_of_chunks=100,              
    min_chunk_size=400,               
)

chunks = semantic_chunk.split_text(text2)

# CHUNK'LARI METADATA İLE BİRLİKTE YÖNET
def process_legal_chunks_with_metadata(chunks):
    """Chunk'ları metadata ile birlikte işle"""
    
    processed_chunks = []
    metadata_chunk = None
    
    for i, chunk in enumerate(chunks):
        # Tarihler chunk'ını tespit et (sadece tarih/rakam içeren)
        word_count = len(chunk.split())
        
        if (word_count <= 5 and 
            all(any(c.isdigit() for c in word) or word in [',', '.'] 
                for word in chunk.split())):
            
            # Bu metadata chunk'ı
            metadata_chunk = {
                'type': 'dates_metadata',
                'content': chunk,
                'dates': chunk.split(','),
                'date_count': len(chunk.split(',')),
                'description': 'Dava sürecindeki kritik tarihler'
            }
            print(f"📅 METADATA CHUNK tespit edildi:")
            print(f"   Tarih sayısı: {metadata_chunk['date_count']}")
            print(f"   İçerik: {chunk[:100]}...")
            
        else:
            # Normal content chunk'ı
            content_info = analyze_chunk_content(chunk, i+1)
            processed_chunks.append({
                'chunk_id': len(processed_chunks) + 1,
                'content': chunk,
                'word_count': word_count,
                'type': content_info['type'],
                'key_elements': content_info['key_elements'],
                'legal_refs': content_info['legal_refs']
            })
    
    return processed_chunks, metadata_chunk

def analyze_chunk_content(chunk, chunk_num):
    """Chunk içeriğini analiz et ve kategorize et"""
    
    chunk_lower = chunk.lower()
    
    # Chunk tipini belirle
    if 'i. dava' in chunk_lower or 'dava dilekçesi' in chunk_lower:
        chunk_type = 'dava_süreci'
    elif 'ii. cevap' in chunk_lower or 'cevap dilekçesi' in chunk_lower:
        chunk_type = 'savunma'
    elif 'iii. mahkeme' in chunk_lower or 'mahkemenin' in chunk_lower:
        chunk_type = 'mahkeme_kararı'
    elif 'bozma' in chunk_lower or 'temyiz' in chunk_lower:
        chunk_type = 'temyiz_süreci'
    elif 'vi. karar' in chunk_lower or 'açıklanan nedenlerle' in chunk_lower:
        chunk_type = 'final_karar'
    else:
        chunk_type = 'genel_içerik'
    
    # Anahtar elemanları tespit et
    key_elements = []
    if 'humk' in chunk_lower:
        key_elements.append('HUMK_referansı')
    if 'ihtiyati tedbir' in chunk_lower:
        key_elements.append('ihtiyati_tedbir')
    if 'tazminat' in chunk_lower:
        key_elements.append('tazminat_talebi')
    if any(date_pattern in chunk for date_pattern in ['.2009', '.2010', '.2011', '.2013', '.2024']):
        key_elements.append('tarih_referansları')
    
    # Hukuki referansları bul
    legal_refs = []
    import re
    
    # Kanun maddesi referansları
    madde_pattern = r'(\d+)\.\s*madde'
    maddeler = re.findall(madde_pattern, chunk_lower)
    if maddeler:
        legal_refs.extend([f"madde_{m}" for m in maddeler])
    
    # Mahkeme dosya numaraları
    dosya_pattern = r'(\d{4}/\d+)'
    dosyalar = re.findall(dosya_pattern, chunk)
    if dosyalar:
        legal_refs.extend([f"dosya_{d}" for d in dosyalar])
    
    return {
        'type': chunk_type,
        'key_elements': key_elements,
        'legal_refs': legal_refs
    }

# KULLANIM VE GÖRÜNTÜLEME
content_chunks, dates_metadata = process_legal_chunks_with_metadata(chunks)

print(f"\n{'='*60}")
print(f"CHUNK ANALIZ RAPORU")
print(f"{'='*60}")

# Metadata bilgisi
if dates_metadata:
    print(f"\n📅 METADATA:")
    print(f"   Tip: {dates_metadata['type']}")
    print(f"   Tarih sayısı: {dates_metadata['date_count']}")
    print(f"   Açıklama: {dates_metadata['description']}")

# Content chunk'ları
print(f"\n📄 İÇERİK CHUNK'LARI ({len(content_chunks)} adet):")

for chunk_info in content_chunks:
    print(f"\n--- CHUNK {chunk_info['chunk_id']} ---")
    print(f"Tip: {chunk_info['type']}")
    print(f"Kelime sayısı: {chunk_info['word_count']}")
    print(f"Anahtar öğeler: {', '.join(chunk_info['key_elements']) if chunk_info['key_elements'] else 'Yok'}")
    print(f"Hukuki referanslar: {', '.join(chunk_info['legal_refs']) if chunk_info['legal_refs'] else 'Yok'}")
    print(f"İçerik önizleme: {chunk_info['content'][:150]}...")

# METADATA'YI KULLANMA ÖRNEKLERİ
def get_case_timeline_from_metadata(dates_metadata):
    """Metadata'dan dava zaman çizelgesini çıkar"""
    if not dates_metadata:
        return []
    
    dates = [date.strip() for date in dates_metadata['content'].split(',')]
    
    # Tarihleri sırala ve kategorize et
    timeline = []
    for date in dates:
        if date:
            timeline.append({
                'date': date,
                'formatted': f"{date[:2]}.{date[2:4]}.{date[4:8]}" if len(date) == 10 else date,
                'category': 'dava_süreci'  # Daha detaylı kategorizasyon yapılabilir
            })
    
    return timeline

def create_searchable_index(content_chunks, dates_metadata):
    """Arama için indeks oluştur"""
    
    search_index = {
        'metadata': dates_metadata,
        'chunks_by_type': {},
        'chunks_with_dates': [],
        'chunks_with_legal_refs': []
    }
    
    # Tip bazında grupla
    for chunk in content_chunks:
        chunk_type = chunk['type']
        if chunk_type not in search_index['chunks_by_type']:
            search_index['chunks_by_type'][chunk_type] = []
        search_index['chunks_by_type'][chunk_type].append(chunk)
        
        # Tarih içeren chunk'lar
        if 'tarih_referansları' in chunk['key_elements']:
            search_index['chunks_with_dates'].append(chunk)
        
        # Hukuki referans içeren chunk'lar
        if chunk['legal_refs']:
            search_index['chunks_with_legal_refs'].append(chunk)
    
    return search_index

# İndeks oluştur
search_index = create_searchable_index(content_chunks, dates_metadata)

print(f"\n📊 ARAMA İNDEKSİ:")
print(f"Chunk tipleri: {list(search_index['chunks_by_type'].keys())}")
print(f"Tarihli chunk'lar: {len(search_index['chunks_with_dates'])}")
print(f"Hukuki referanslı chunk'lar: {len(search_index['chunks_with_legal_refs'])}")

# ÖRNEK SORGULAR
def query_chunks(search_index, query_type, query_value=None):
    """Chunk'larda arama yap"""
    
    if query_type == 'by_type':
        return search_index['chunks_by_type'].get(query_value, [])
    
    elif query_type == 'with_dates':
        return search_index['chunks_with_dates']
    
    elif query_type == 'with_legal_refs':
        return search_index['chunks_with_legal_refs']
    
    elif query_type == 'metadata':
        return search_index['metadata']
    
    return []

# Kullanım örnekleri:
print(f"\n🔍 ÖRNEK SORGULAR:")
print(f"Mahkeme kararı chunk'ları: {len(query_chunks(search_index, 'by_type', 'mahkeme_kararı'))}")
print(f"Tarihli chunk'lar: {len(query_chunks(search_index, 'with_dates'))}")
print(f"Metadata: {query_chunks(search_index, 'metadata')['type'] if query_chunks(search_index, 'metadata') else 'Yok'}")


CHUNK ANALIZ RAPORU

📄 İÇERİK CHUNK'LARI (48 adet):

--- CHUNK 1 ---
Tip: dava_süreci
Kelime sayısı: 1058
Anahtar öğeler: HUMK_referansı, ihtiyati_tedbir, tazminat_talebi, tarih_referansları
Hukuki referanslar: madde_109, madde_110, madde_109, madde_109, madde_109, madde_109, madde_109, dosya_2023/596, dosya_2024/257, dosya_2022/3281, dosya_2024/117, dosya_2009/139, dosya_2010/27, dosya_2010/695, dosya_2011/916, dosya_2009/139, dosya_2013/2984, dosya_2013/3773, dosya_2009/139, dosya_2010/27
İçerik önizleme: 6. Hukuk Dairesi 2023/596 E. , 2024/257 K. \n "İçtihat Metni" MAHKEMESİ : ... Bölge Adliye Mahkemesi 13. Hukuk Dairesi Taraflar arasındaki rücuen tazm...

--- CHUNK 2 ---
Tip: mahkeme_kararı
Kelime sayısı: 454
Anahtar öğeler: HUMK_referansı, ihtiyati_tedbir, tazminat_talebi, tarih_referansları
Hukuki referanslar: madde_109, madde_109, dosya_2009/139
İçerik önizleme: 1. Asliye Hukuk Mahkemesinin 2009/139 D. ... sayılı dosyası ile dosya kapsamı birlikte değerlendirildiğinde; davalını